Notes for tomorrow...

I think it's a little too complex to use word vectors from SpaCy with TensorFlow/Keras for this course. It isn't hard, just more work than we should expect students to do. And too advanced for them without some deep learning & programming skills.

So. I think maybe we shouldn't use SpaCy here and instead learn the embeddings so we can use TF for the entire thing. 

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

#import spacy

# Need to load the large model to get the vectors
#nlp = spacy.load('en_core_web_lg')

In [7]:
from sklearn.model_selection import train_test_split

# Load in the data
review_data = pd.read_csv('../input/yelp_ratings.csv', index_col=0)
review_data.head()
vectors = np.load('../input/review_vectors.npy')

X_train, X_test, y_train, y_test = train_test_split(vectors, review_data.sentiment.values, 
                                                    test_size=0.1, random_state=1)

### Exercise: Create a dense network

In [51]:
tf.random.set_seed(1)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [52]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [53]:
history = model.fit(x=X_train, y=y_train, epochs=10, validation_split=0.1)

Train on 36071 samples, validate on 4008 samples
Epoch 1/10
36071/36071 [==============================] - 3s 88us/sample - loss: 0.2759 - accuracy: 0.8838 - val_loss: 0.1996 - val_accuracy: 0.9202
Epoch 2/10
36071/36071 [==============================] - 2s 59us/sample - loss: 0.1982 - accuracy: 0.9228 - val_loss: 0.1987 - val_accuracy: 0.9199
Epoch 3/10
36071/36071 [==============================] - 2s 59us/sample - loss: 0.1871 - accuracy: 0.9263 - val_loss: 0.1815 - val_accuracy: 0.9274
Epoch 4/10
36071/36071 [==============================] - 2s 59us/sample - loss: 0.1825 - accuracy: 0.9277 - val_loss: 0.1729 - val_accuracy: 0.9346
Epoch 5/10
36071/36071 [==============================] - 2s 59us/sample - loss: 0.1787 - accuracy: 0.9303 - val_loss: 0.1833 - val_accuracy: 0.9291
Epoch 6/10
36071/36071 [==============================] - 2s 63us/sample - loss: 0.1751 - accuracy: 0.9312 - val_loss: 0.1740 - val_accuracy: 0.9326
Epoch 7/10
36071/36071 [==============================] -

In [54]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(accuracy)

0.9463404


Without much effort, your neural network has better accuracy than the logistic regression model and the SVC model. You used the document vectors for this. Potentially you could do better by using embeddings for each word rather than the averaged document vector.

### Tokenize and encode review text with Keras

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [12]:
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')

In [13]:
tokenizer.fit_on_texts(review_data.text)

In [24]:
# create an array of zeros for filling with encoded tokens
encoded_text = np.zeros((len(review_data.text), 100), dtype=int)

In [25]:
for i, sequence in enumerate(tokenizer.texts_to_sequences_generator(review_data.text)):
    if sequence:
        encoded_text[i, -len(sequence):] = sequence[:100]

In [27]:
encoded_text.shape

(44533, 100)

### Exercise: Define the model

If you didn't already have word embeddings from SpaCy, you'd use an `Embedding` layer here.

In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.num_words, 300),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [29]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
history = model.fit(x=encoded_text, y=review_data.sentiment.values, epochs=10)

Train on 44533 samples
Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
44533/44533 [==============================] - 226s 5ms/sample - loss: 0.2407 - accuracy: 0.9040
Epoch 2/10
44533/44533 [==============================] - 195s 4ms/sample - loss: 0.1316 - accuracy: 0.9512
Epoch 3/10
44533/44533 [==============================] - 195s 4ms/sample - loss: 0.0886 - accuracy: 0.9677
Epoch 4/10
 1760/44533 [>.............................] - ETA: 3:41 - loss: 0.0479 - accuracy: 0.9850

KeyboardInterrupt: 

Definitely going to need to use a GPU for this....

In [31]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(accuracy)

InvalidArgumentError:  indices[22,140] = -1 is not in [0, 10000)
	 [[node sequential/embedding/embedding_lookup (defined at /Users/mat/miniconda3/envs/kaggle/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_16818]

Function call stack:
distributed_function
